# Python Closures, Decorators, Generators & Scope (Local / Global / Nonlocal)

This notebook is a **complete practical guide** with **multiple examples** and clear explanations.

## Topics
- Scope rules: Local, Global, Nonlocal (LEGB)
- Closures (what/why)
- Decorators (what/why) + `functools.wraps`
- Decorators with args, stacking, and class decorators
- Generators (`yield`) + generator expressions
- Advanced generators: `send`, `throw`, `close`
- Tricky pitfalls and interview-style questions


## 1) Scope in Python: LEGB Rule

Python resolves variables using **LEGB** order:

1. **L — Local**: inside the current function
2. **E — Enclosing**: inside outer function(s) (used in closures)
3. **G — Global**: at the module/file level
4. **B — Built-in**: Python built-ins like `len`, `sum`, etc.


In [ ]:
# LEGB demo
x = "GLOBAL x"

def outer():
    x = "ENCLOSING x"
    def inner():
        x = "LOCAL x"
        print(x)  # Local
    inner()
    print(x)      # Enclosing

outer()
print(x)          # Global


## 2) Local vs Global: `global` keyword

### Key idea
- If you **assign** to a variable inside a function, Python treats it as **local** by default.
- If you want to modify a global variable from inside a function, use `global`.

⚠️ In real projects, overusing `global` is discouraged (harder to debug).

In [ ]:
count = 0

def increment_wrong():
    # Python thinks 'count' is local because we assign to it
    # Uncommenting the next line will raise UnboundLocalError
    # count += 1
    pass

def increment_global():
    global count
    count += 1

increment_global()
increment_global()
print("count =", count)


## 3) Enclosing scope: `nonlocal` keyword

### Key idea
- Use `nonlocal` to modify a variable from an **enclosing (outer) function**.
- This is very common in **closures**.

✅ `nonlocal` works only with variables in the enclosing function scope (not global).

In [ ]:
def outer_counter():
    n = 0
    def inc():
        nonlocal n
        n += 1
        return n
    return inc

c = outer_counter()
print(c())
print(c())
print(c())


# ✅ Closures

## 4) What is a Closure?

A **closure** happens when:
- An inner function **remembers** variables from an outer function
- Even after the outer function has finished executing

### Why closures matter
- Great for **private state** (like counters)
- Useful for **factory functions** (functions that generate other functions)
- Used heavily in decorators


In [ ]:
# Closure: function factory
def power_factory(exp):
    # exp is captured by the inner function
    def power(base):
        return base ** exp
    return power

square = power_factory(2)
cube = power_factory(3)

print("square(5) =", square(5))
print("cube(5)   =", cube(5))


## 5) Closure with private state (counter)

This is a common real-world pattern: encapsulating state without a class.

In [ ]:
def make_counter(start=0):
    n = start
    def next_value():
        nonlocal n
        n += 1
        return n
    return next_value

c1 = make_counter(100)
c2 = make_counter(0)

print(c1(), c1(), c1())  # 101 102 103
print(c2(), c2())        # 1 2


## 6) Tricky Closure Pitfall: late binding in loops

Problem:
- Closures capture variables by **reference**, not by value.
- In loops, you might expect each function to remember a different loop value, but they may all share the last value.

✅ Fix: use default argument `i=i`.


In [ ]:
funcs = []
for i in range(3):
    def f():
        return i
    funcs.append(f)

print([fn() for fn in funcs])  # Often surprises: [2, 2, 2]

# Fix using default argument
fixed_funcs = []
for i in range(3):
    def f(i=i):
        return i
    fixed_funcs.append(f)

print([fn() for fn in fixed_funcs])  # [0, 1, 2]


# ✅ Decorators

## 7) What is a Decorator?

A **decorator** is a function that:
- Takes another function as input
- Returns a new function (wrapped/enhanced version)

### Why decorators matter
- Logging
- Timing / performance measurement
- Access control
- Caching
- Validation

### Key syntax
```python
@decorator
def my_func(...):
    ...
```


In [ ]:
# Basic decorator
def simple_decorator(func):
    def wrapper():
        print("[Before] calling", func.__name__)
        result = func()
        print("[After] called", func.__name__)
        return result
    return wrapper

@simple_decorator
def hello():
    print("Hello!")

hello()


## 8) Decorators with arguments (`*args`, `**kwargs`)

If your function takes parameters, the wrapper must accept them.

### Rule
- Use `*args` for positional args
- Use `**kwargs` for keyword args


In [ ]:
def log_calls(func):
    def wrapper(*args, **kwargs):
        print("Calling", func.__name__, "args=", args, "kwargs=", kwargs)
        result = func(*args, **kwargs)
        print("Returned:", result)
        return result
    return wrapper

@log_calls
def add(a, b):
    return a + b

@log_calls
def greet(name, greeting="Hi"):
    return f"{greeting}, {name}!"

print(add(10, 20))
print(greet("Anshu"))
print(greet("Anshu", greeting="Hello"))


## 9) The `functools.wraps` requirement (IMPORTANT)

Without `wraps`, your decorated function loses metadata:
- `__name__`
- docstring
- signature

✅ Use `@wraps(func)` inside your decorator.


In [ ]:
from functools import wraps

def logger(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("[LOG]", func.__name__, "called")
        return func(*args, **kwargs)
    return wrapper

@logger
def multiply(a, b):
    """Multiply two numbers."""
    return a * b

print(multiply(3, 4))
print("Name:", multiply.__name__)
print("Doc:", multiply.__doc__)


## 10) Decorator Factory (decorator with its own parameters)

Sometimes you want:
- `@retry(3)`
- `@timeit(unit="ms")`

That means the decorator itself must be created by another function.

In [ ]:
from functools import wraps
import time

def timeit(unit="ms"):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            start = time.perf_counter()
            result = func(*args, **kwargs)
            end = time.perf_counter()

            elapsed_s = end - start
            if unit == "ms":
                print(f"{func.__name__} took {elapsed_s * 1000:.3f} ms")
            else:
                print(f"{func.__name__} took {elapsed_s:.6f} s")
            return result
        return wrapper
    return decorator

@timeit(unit="ms")
def slow_add(a, b):
    time.sleep(0.05)
    return a + b

print(slow_add(10, 20))


## 11) Stacking Decorators (execution order)

Decorators apply **bottom-up**, but execute like nested wrappers.

```python
@A
@B
def f():
    pass
```
is equivalent to:
```python
f = A(B(f))
```


In [ ]:
from functools import wraps

def dec_a(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("A before")
        out = func(*args, **kwargs)
        print("A after")
        return out
    return wrapper

def dec_b(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print("B before")
        out = func(*args, **kwargs)
        print("B after")
        return out
    return wrapper

@dec_a
@dec_b
def greet():
    print("Hello")

greet()


## 12) Class-based decorator (advanced)

A class can be used as a decorator by implementing `__call__`.


In [ ]:
from functools import wraps

class CountCalls:
    def __init__(self, func):
        wraps(func)(self)
        self.func = func
        self.calls = 0

    def __call__(self, *args, **kwargs):
        self.calls += 1
        print(f"{self.func.__name__} call #{self.calls}")
        return self.func(*args, **kwargs)

@CountCalls
def say(msg):
    return msg.upper()

print(say("hello"))
print(say("world"))


# ✅ Generators

## 13) What is a Generator?

A generator is a function that uses `yield` instead of `return`.

### Key idea
- `return` finishes the function
- `yield` pauses and remembers state, resuming from there next time

### Why generators are important
- Memory-efficient (process big data line-by-line)
- Enables pipeline-style processing (very data engineering friendly)


In [ ]:
def simple_gen():
    yield 1
    yield 2
    yield 3

g = simple_gen()
print(next(g))
print(next(g))
print(next(g))

try:
    print(next(g))
except StopIteration:
    print("Generator finished")


## 14) Generator vs List: memory behavior

List comprehension builds the whole list immediately.
Generator expression computes items lazily.


In [ ]:
nums_list = [x * x for x in range(5)]  # builds list now
nums_gen = (x * x for x in range(5))   # generator (lazy)

print("list:", nums_list)
print("gen:", nums_gen)
print("consume gen:", list(nums_gen))
print("consume again:", list(nums_gen))  # empty after consumed


## 15) Generator pipeline example (filter + transform)

This is a very FP-style way to process data.

We will:
- generate numbers
- filter even
- square them


In [ ]:
def numbers(n):
    for i in range(1, n + 1):
        yield i

evens = (x for x in numbers(10) if x % 2 == 0)
squares = (x * x for x in evens)

print(list(squares))


## 16) `send()` (advanced generator)

`send(value)` resumes the generator and sends a value inside.

⚠️ First call must be `next(gen)` or `gen.send(None)`.


In [ ]:
def echo():
    value = yield "ready"  # pause here and receive a value later
    while True:
        value = yield f"you sent: {value}"

g = echo()
print(next(g))          # starts generator, returns 'ready'
print(g.send(10))       # sends 10
print(g.send("hello"))  # sends 'hello'


## 17) `throw()` and `close()` (advanced generator)

- `throw(ExceptionType)` raises an exception inside generator
- `close()` stops the generator


In [ ]:
def safe_counter():
    i = 0
    try:
        while True:
            yield i
            i += 1
    except ValueError:
        yield "ValueError handled inside generator"

g = safe_counter()
print(next(g))
print(next(g))
print(g.throw(ValueError))
g.close()

try:
    print(next(g))
except StopIteration:
    print("Generator closed")


# ✅ Combined Example: Closures + Decorators + Scope

We will build a decorator that counts calls using a closure.

This uses:
- closure state
- `nonlocal`
- decorator wrapping


In [ ]:
from functools import wraps

def count_calls():
    calls = 0  # enclosing variable
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            nonlocal calls
            calls += 1
            print(f"{func.__name__} called {calls} times")
            return func(*args, **kwargs)
        return wrapper
    return decorator

@count_calls()
def work(x):
    return x * 2

print(work(5))
print(work(8))


# ✅ Tricky Questions / Pitfalls (Must Know)

## 1) Why does assigning inside function create a local variable?
- Because Python decides scope at compile time.

## 2) Closure late-binding in loops
- All closures may capture the same final loop variable.
- Fix with `i=i` default argument.

## 3) Decorator metadata loss
- Without `wraps`, the function name/docstring changes.

## 4) Generator exhaustion
- Once consumed, it's empty.

## 5) `send()` first call
- First resume must be `next()` or `.send(None)`.


---
## ✅ Final Recap

- **Closures**: inner functions remember enclosing variables.
- **Decorators**: functions that wrap/enhance other functions.
- **Generators**: lazy iterators using `yield`.
- **Scope**: LEGB + `global` and `nonlocal` control assignment scope.
